## Extract image features

In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from PIL import Image

# Load pre-trained ResNet model and remove the top fully connected layer
base_model = resnet18(pretrained=True)
base_model = torch.nn.Sequential(*(list(base_model.children())[:-1]))

batch = "4cbatch1"

# Define image preprocessing
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # the mean and std come from imagenet dataset
])

# Define dataset folder path
trainset = f'dataset/{batch}_train'
valset = f'dataset/{batch}_val'
testset = f'dataset/{batch}_test'

# Create an empty list to store features
train_features = []
val_features = []
test_features = []

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(trainset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(trainset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        train_features.append(features.squeeze())


# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(valset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(valset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        val_features.append(features.squeeze())

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(testset)):
    if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
        # Construct image path
        image_path = os.path.join(testset, filename)

        # Load image
        img = Image.open(image_path)

        # Preprocess image and add batch dimension
        img_tensor = preprocess(img).unsqueeze(0)

        # Set model to evaluation mode
        base_model.eval()

        # Use ResNet model for feature extraction
        with torch.no_grad():
            features = base_model(img_tensor)

        # Append features to the list
        test_features.append(features.squeeze())


c:\Users\zl23i\.conda\envs\avocado\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\zl23i\.conda\envs\avocado\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Read labals 

In [2]:
import os
import pandas as pd

# Load label.csv file
label_df = pd.read_csv(f'label/{batch}label.csv')
labelname = "Firmness"

# Initialize lists to store labels
train_labels = []
val_labels = []
test_labels = []

# Iterate through all image files in the dataset folder
for filename in sorted(os.listdir(trainset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename.replace("original_", ""))[0]

        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        train_labels.append(label)

for filename in sorted(os.listdir(valset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename)[0]
        
        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        val_labels.append(label)

for filename in sorted(os.listdir(testset)):
    if filename.endswith('.jpg'):
        # Extract filename without extension
        filename_without_extension = os.path.splitext(filename)[0]
        
        # Extract label from label.csv based on filename
        label = label_df[label_df["Image_name"] == filename_without_extension][labelname].iloc[0]
        test_labels.append(label)

train_labels = [torch.tensor([float(train_labels[i])]) for i in range(len(train_labels))]
val_label = [torch.tensor([float(val_labels[i])]) for i in range(len(val_labels))]
test_label = [torch.tensor([float(test_labels[i])]) for i in range(len(test_labels))]

train_features_tensor = torch.stack(train_features)
train_labels_tensor = torch.tensor(train_labels)
val_features_tensor = torch.stack(val_features)
val_labels_tensor = torch.tensor(val_labels)
test_features_tensor = torch.stack(test_features)
test_labels_tensor = torch.tensor(test_labels)


## Random forest

In [3]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import numpy as np

# Define the random forest model
# 定义随机森林模型
model = RandomForestRegressor(random_state=42)

# Define the parameter grid for hyperparameter tuning
# 定义参数空间
param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4]  # Minimum number of samples required to be at a leaf node
}

# Create a GridSearchCV object with parallel computation
# 创建 GridSearchCV 对象，并设置并行计算
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
# 拟合网格搜索模型
grid_search.fit(train_features_tensor, train_labels_tensor)

# Output the best parameter combination
# 输出最佳参数组合
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
# 使用最佳参数训练模型
best_model = grid_search.best_estimator_
best_model.fit(train_features_tensor, train_labels_tensor)

# Make predictions on the training and validation sets
# 在训练集和验证集上进行预测
train_predictions = best_model.predict(train_features_tensor)
val_predictions = best_model.predict(val_features_tensor)

# Calculate mean squared error and R^2 score
# 计算均方误差和 R^2 分数
train_loss = mean_squared_error(train_labels_tensor, train_predictions)
val_loss = mean_squared_error(val_labels_tensor, val_predictions)
train_r2 = r2_score(train_labels_tensor, train_predictions)
val_r2 = r2_score(val_labels_tensor, val_predictions)

# Output performance metrics on the training and validation sets
# 输出训练和验证集上的性能指标
print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# Make predictions on the test set
# 在测试集上进行预测
test_predictions = best_model.predict(test_features_tensor)
mse = mean_squared_error(test_labels_tensor, test_predictions)
r_squared = r2_score(test_labels_tensor, test_predictions)

# Output predictions and performance metrics on the test set
# 输出测试集上的预测结果和性能指标
print("Predictions:", test_predictions)
print("True Labels:", test_labels_tensor.tolist())
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R-squared: {r_squared:.4f}')

Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Train Loss: 8.4684, Validation Loss: 39.2031
Train R-squared: 0.9220, Validation R-squared: 0.6389
Predictions: [26.57699714 25.57539434 37.74185804 29.51584759 32.23375918 52.41146761
 42.13057573 36.60850375 28.54787805 28.21525581 30.08454622 25.94179022
 29.38299835 27.37981849 27.08488426 29.96574356 27.27615799 27.93112162
 28.47924814 29.53597275 26.53418472 27.05565574 25.30514231 41.00318572
 26.75359762 24.50888723 28.14593162 27.09053808 25.7063366  26.03166824
 23.75860982 25.19281562 23.60129978 27.27844381 23.98682136 24.74860358
 24.62206988 25.79121653 24.05937603 40.00160645 26.66610993 21.67446502
 22.44397955 27.17205305 20.8974522  19.95687197 21.26004222 20.55854148
 20.97670439 23.82358528]
True Labels: [32.2687, 39.0408, 26.6264, 25.9661, 40.188, 52.5866, 39.2094, 68.8597, 41.964, 32.8633, 25.93395, 22.343725, 59.33145, 32.5974, 22.957475, 31.137125, 26.165675

## SVR

In [5]:
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

# Define the SVR model
# 定义 SVR 模型
svr = SVR()

# Define the parameter search space
# 定义参数搜索范围
param_grid = {
    'kernel': ['linear', 'poly', 'rbf'],  # Kernel types
    'C': [0.01, 0.1, 1, 10, 100],        # Regularization parameter
    'epsilon': [0.01, 0.1, 1, 10]        # Epsilon in the epsilon-SVR model
}

# Create the GridSearchCV object with parallel computation
# 创建 GridSearchCV 对象，并设置并行计算
grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
# 拟合网格搜索模型
grid_search.fit(train_features_tensor, train_labels_tensor)

# Output the best parameter combination
# 输出最佳参数组合
print("Best Parameters:", grid_search.best_params_)

# Train the model with the best parameters
# 使用最佳参数训练模型
best_model = grid_search.best_estimator_
best_model.fit(train_features_tensor, train_labels_tensor)

# Make predictions on the training and validation sets
# 在训练集和验证集上进行预测
train_predictions = best_model.predict(train_features_tensor)
val_predictions = best_model.predict(val_features_tensor)

# Calculate mean squared error and R^2 score
# 计算均方误差和 R^2 分数
train_loss = mean_squared_error(train_labels_tensor, train_predictions)
val_loss = mean_squared_error(val_labels_tensor, val_predictions)
train_r2 = r2_score(train_labels_tensor, train_predictions)
val_r2 = r2_score(val_labels_tensor, val_predictions)

# Output performance metrics on the training and validation sets
# 输出训练和验证集上的性能指标
print(f'Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
print(f'Train R-squared: {train_r2:.4f}, Validation R-squared: {val_r2:.4f}')

# Make predictions on the test set
# 在测试集上进行预测
test_predictions = best_model.predict(test_features_tensor.numpy())
test_labels = test_labels_tensor.numpy()

# Calculate mean squared error and R^2 score on the test set
# 计算测试集上的均方误差和 R^2 分数
mse = mean_squared_error(test_labels, test_predictions)
r_squared = r2_score(test_labels, test_predictions)

# Output predictions and performance metrics on the test set
# 输出测试集上的预测结果和性能指标
print("Predictions:", test_predictions)
print("True Labels:", test_labels)
print(f'Test RMSE: {np.sqrt(mse):.4f}')
print(f'Test R-squared: {r_squared:.4f}')

Best Parameters: {'C': 100, 'epsilon': 1, 'kernel': 'poly'}
Train Loss: 2.5369, Validation Loss: 37.5164
Train R-squared: 0.9766, Validation R-squared: 0.6544
Predictions: [27.97147416 32.14217975 34.77594768 30.95234587 36.67550551 49.41387455
 45.71816873 53.80917488 32.72595207 26.93663296 27.55328553 28.50211757
 38.5117562  29.6333306  32.285926   30.47783819 32.97092012 24.97484571
 35.34429508 38.26014794 26.65211664 24.78231801 30.62541175 37.94166253
 24.36867533 27.32196776 22.79112672 33.58403594 10.51968719 21.82248223
 22.21210059 20.19090886 24.2054041  29.52756952 22.90403096 21.54840843
 19.90125224 27.76926556 21.6035074  36.93585288 34.39074255 18.76414024
 18.83173826 33.44960381 21.62208371 19.70475584 24.50789122 20.98766076
 20.12129406 16.78646572]
True Labels: [32.2687   39.0408   26.6264   25.9661   40.188    52.5866   39.2094
 68.8597   41.964    32.8633   25.93395  22.343725 59.33145  32.5974
 22.957475 31.137125 26.165675 26.841175 24.381675 38.512625 24.816